AutoGen Core

Something a little different.

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

The fundamental principle

Autogen Core decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a Runtime.

There are 2 types: **Standalone** and **Distributed**.

Standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.


In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)

True

Define a Message Onject(whatever structure we want for messages in our Agent Framework)

In [2]:
# simple
@dataclass
class Message:
    content: str

Define an Agent. It is a subclass of RoutedAgent.
Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [3]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.")
        

Standalone runtime and register an agent type

In [4]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

In [5]:
# start a runtime and send a message
runtime.start()

In [6]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi here!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi here!' and I disagree.


In [7]:
await runtime.stop()
await runtime.close()

create AgentChat Assistant

In [8]:
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [9]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [10]:
runtime.start()  # Start processing messages in the background.
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: Hi there!
LLMAgent responded: Hello! How can I assist you today?
>>> Hello! How can I assist you today?
>>> This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent responded: I understand you have a different opinion. Please let me know how you'd like to proceed or if there's something specific you'd like to discuss!


In [11]:
await runtime.stop()
await runtime.close()

Play game(rock, paper, scissors) between 3 agents

In [12]:
from autogen_ext.models.ollama import OllamaChatCompletionClient

# player 1 agent
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
# player 2 agent
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

Judge agent

In [13]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message, inner_1)
        response2 = await self.send_message(message, inner_2)
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [14]:
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [15]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock
Player 2: scissors
Player 1 wins because rock beats scissors. TERMINATE


In [16]:
await runtime.stop()
await runtime.close()

Quick intro to AutoGen Core - Distributed


In [ ]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

# change here to false if want to have multiple workes(see below)
ALL_IN_ONE_WORKER = False

In [18]:
# message class
@dataclass
class Message:
    content: str

In [19]:
# A host for our distributed runtime
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [20]:
# use serper tool again for search
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [21]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [22]:
# Create Agents
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [ ]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    
    agent_id = AgentId("judge", "default")




In [24]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [25]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of choosing AutoGen for your new AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen excels in orchestrating multi-agent systems, allowing agents to communicate and cooperate effectively through natural conversations. This facilitates complex problem-solving and enhances overall efficiency.

2. **Flexibility and Customization**: The framework is designed to be highly flexible, enabling developers to tailor agents for specific tasks easily. This customization allows for the creation of specialized AI systems that can adapt to various needs.

3. **Reduced Coordination Complexity**: By eliminating the need for custom inter-agent protocols, AutoGen simplifies the coordination of agent interactions, making it easier to integrate and manage multiple agents in a project.

4. **Enhanced Efficiency**: AutoGen helps reduce the need for human intervention in repetitive and complex tasks, which leads to faster execution and better resource management.

5. **Support for Autonomous Operations**: The platform supports both fully autonomous operation of agents and human-in-the-loop systems, providing versatility in how AI can be deployed.

6. **Focus on Orchestration**: Unlike traditional structured flowcharts, AutoGen emphasizes conversational strategies for managing agent interactions, which can provide a more intuitive development process.

7. **Improved Factuality and Reasoning**: Through collaborative dialogues among agents, AutoGen enhances the quality of output by improving factual accuracy and reasoning capabilities.

Overall, AutoGen offers a comprehensive framework that is beneficial for building scalable, efficient, and intelligent multi-agent systems.

TERMINATE

## Cons of AutoGen:
Here are some notable cons of using AutoGen in an AI Agent project:

1. **Complex Documentation**: The documentation for AutoGen can be hard to read and lacks sufficient examples, making it difficult for new users to understand and implement effectively.

2. **Limited Compatibility**: AutoGen performs optimally with the latest large language models, which may lead to issues when used with less powerful models, potentially causing performance problems.

3. **Steep Learning Curve**: Its graph-based approach and the necessity to understand Directed Acyclic Graphs (DAGs) and state management can be challenging for new developers, resulting in a higher learning curve.

4. **Deployment Challenges**: AutoGen requires significant customization for deployment and may necessitate infrastructure management, which can complicate the setup process for teams devoid of strong engineering resources.

5. **Lack of Intuition for Simple Tasks**: For straightforward or basic tasks, AutoGen might be seen as overkill, as its strengths lie in adapting to complex workflows rather than handling simple operations efficiently.

6. **Potential for Bugs**: Users have reported issues where certain features, such as structured outputs, do not work as intended, impacting reliability.

7. **Security Concerns**: There are limited mentions of critical security features such as data encryption or intellectual property control, which could pose risks depending on the project requirements.

These factors may influence the decision on whether AutoGen is the right choice for specific AI agent projects. 

TERMINATE



## Decision:

Based on the thorough evaluation of the pros and cons of AutoGen, I would recommend proceeding with the use of AutoGen for the project.

**Rationale**: The advantages of AutoGen, particularly its ability to facilitate complex multi-agent collaboration, enhance efficiency, and support autonomous operations, align well with the needs of modern AI projects. While the documentation complexity and learning curve pose challenges, these can be mitigated through targeted training and resources. The potential issues with compatibility and deployment should also be weighed against the significant benefits that AutoGen provides in orchestrating intelligent, flexible systems capable of complex problem-solving.

TERMINATE

In [26]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()


await host.stop()